In [5]:
import pandas as pd
import numpy as np

import binascii

In [144]:
hex_to_bin = {
    "0": "0000",
    "1": "0001",
    "2": "0010",
    "3": "0011",
    "4": "0100",
    "5": "0101",
    "6": "0110",
    "7": "0111",
    "8": "1000",
    "9": "1001",
    "A": "1010",
    "B": "1011",
    "C": "1100",
    "D": "1101",
    "E": "1110",
    "F": "1111",
}

In [42]:
HEADER = [
    {"size": 3, "indent": 0, "name": "version"},
    {"size": 3, "indent": 3, "name": "type_id"},
]

In [175]:
def decode_type_4(data, info):
    """ type_id 4 is a literal """
    
    litteral = ""
    for i in range((len(data)-6)//5):
        litteral += data[3:][3:][i*5:][1:][:4]
        if data[3:][3:][i*5:][:1] == 0:
            break
    
    info["litteral"] = int(litteral,2)
    print(info)
    
    return info

In [176]:
def decode_length_type_0(data, info):
    """ length_type 0 """
    
    info["total_length_subpackets"] = int(data[3:][3:][1:][:15],2)
    info["subpackets_bin"] = []
    info["subpackets_bin"].append(data[3:][3:][1:][15:][:11])
    info["subpackets_bin"].append(data[3:][3:][1:][15:][11:][:info["total_length_subpackets"]-11])
    
    
    info["subpackets"] = []
    for packet in info["subpackets_bin"]:
        sub_info = decode_subpackets(packet)
        info["subpackets"].append(sub_info)
        
    print("length type 0", info)
    return info

def decode_length_type_1(data, info):
    """ length_type 1 """
    
    info["num_subpackets"] = int(data[3:][3:][1:][:11],2)
    info["subpackets_bin"] = []
    for packet in range(info["num_subpackets"]):
        info["subpackets_bin"].append(data[3:][3:][1:][11*i:][:11])
    
    info["subpackets"] = []
    for packet in info["subpackets_bin"]:
        sub_info = decode_subpackets(packet)
        info["subpackets"].append(sub_info)
        
    #print("length type 1", info)
    return info    

In [177]:
def decode_subpackets(packet):
    
    sub_info = {}
    for item in HEADER:
        #print("indent", type(item["indent"]), "size", type(item["size"]))
        sub_info[item["name"]] = int(packet[item["indent"]:][:item["size"]],2)
    
    if sub_info["type_id"] == 4:
        sub_info = decode_type_4(packet, sub_info)
    if sub_info["type_id"] != 4:
        sub_info["length_id"] = packet[3:][3:][:1]
        if sub_info["length_id"] == 0:
            sub_info = decode_length_type_0(packet, sub_info)
        elif sub_info["length_id"] == 1:
            sub_info = decode_length_type_1(packet, sub_info)
            
    return sub_info

In [178]:
def compute_the_thing(input_file="test_input.txt"):
    
    with open(input_file, 'r') as f:
        data = f.readlines()
        
    code = [line.strip("\n") for line in data][0]
    data = ""
    
    for char in code:
        data += hex_to_bin[char]
    
    print(code)
    print(data)
    
    info = {}
    for item in HEADER:
        print(item["name"], int(data[item["indent"]:][:item["size"]],2))
        info[item["name"]] = int(data[item["indent"]:][:item["size"]],2)
        
    if info["type_id"] == 4:
        info = decode_type_4(data, info)
        
    if (info["type_id"] != 4):
        info["length_id"] = int(data[3:][3:][:1])
        if info["length_id"] == 0:
            info = decode_length_type_0(data, info)
        elif info["length_id"] == 1:
            info = decode_length_type_1(data, info)
    
    return info

In [179]:
info = compute_the_thing("test_input_example.txt")

D2FE28
110100101111111000101000
version 6
type_id 4
{'version': 6, 'type_id': 4, 'litteral': 2021}


In [183]:
info = compute_the_thing("test_input2.txt")

38006F45291200
00111000000000000110111101000101001010010001001000000000
version 1
type_id 6
{'version': 6, 'type_id': 4, 'litteral': 10}
{'version': 2, 'type_id': 4, 'litteral': 20}
length type 0 {'version': 1, 'type_id': 6, 'length_id': 0, 'total_length_subpackets': 27, 'subpackets_bin': ['11010001010', '0101001000100100'], 'subpackets': [{'version': 6, 'type_id': 4, 'litteral': 10}, {'version': 2, 'type_id': 4, 'litteral': 20}]}


In [184]:
info = compute_the_thing("test_input3.txt")

EE00D40C823060
11101110000000001101010000001100100000100011000001100000
version 7
type_id 3


NameError: name 'i' is not defined